Multi  Class

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim


from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [9]:
# יצירת דאטה מדומה
X = torch.rand(100, 10)  # 100 דוגמאות עם 10 פיצ'רים כל אחת
y = torch.randint(0, 2, (100, 4)).float()  # 4 תוויות שונות לכל דגימה (0 או 1 לכל תווית)


torch.Size([100, 4])

In [24]:
y.shape

torch.Size([100, 4])

In [7]:
import pandas as pd

# יצירת DataFrame
df = pd.DataFrame(torch.cat((X, y), dim=1).numpy(),
                  columns=[f"Feature_{i+1}" for i in range(10)]
                  + [f"Label_{i+1}" for i in range(4)])



In [8]:
df.head()

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Label_1,Label_2,Label_3,Label_4
0,0.131714,0.632056,0.604103,0.840977,0.726983,0.140683,0.905966,0.121806,0.703736,0.525937,1.0,0.0,0.0,0.0
1,0.846537,0.882095,0.830401,0.101642,0.591251,0.928121,0.220322,0.990815,0.342939,0.163724,0.0,1.0,1.0,0.0
2,0.232560,0.939241,0.837838,0.239936,0.489377,0.188696,0.693155,0.561615,0.129677,0.391103,0.0,1.0,0.0,1.0
3,0.171398,0.916550,0.927355,0.067276,0.232505,0.040732,0.282172,0.278982,0.123086,0.727525,0.0,0.0,1.0,1.0
4,0.292320,0.057460,0.904620,0.119409,0.920117,0.113558,0.977570,0.176744,0.545542,0.320785,1.0,0.0,0.0,0.0


In [10]:
# פיצול לנתוני אימון ובדיקה
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                     random_state=42)

# המרת הנתונים ל-Tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

<ipython-input-10-53adaae51e1b>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
<ipython-input-10-53adaae51e1b>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
<ipython-input-10-53adaae51e1b>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
<ipython-input-10-53adaae51e1b>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clon

In [11]:
X.shape[1]

10

In [12]:
 y.shape[1]

4

In [13]:
input_dim = X.shape[1]
num_labels = y.shape[1]

In [23]:
#הגדרת מודל
model = nn.Sequential(
        nn.Linear(input_dim, 32),
        nn.ReLU(),
        nn.Linear(32,16),
        nn.Tanh(),
        nn.Linear(16,num_labels),
        nn.Sigmoid()
)

# הגדרת פונקציית הפסד ואופטימיזציה
criterion = nn.BCELoss()  # פונקציית הפסד לבינארי (Binary Cross-Entropy)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# אימון המודל
epochs =1000
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    if epoch%111 == 0:
      print(loss.item())


0.6870844960212708
0.22205372154712677
0.035210639238357544
0.010362349450588226
0.0047183106653392315
0.0027298175264149904
0.0018030398059636354
0.0012928057694807649
0.0009777855593711138
0.0007675917586311698


In [22]:
from sklearn.metrics import classification_report

# חיזוי על נתוני הבדיקה
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    predictions = (test_outputs > 0.5).float()  # הפיכת הסתברויות ל-0/1

# יצירת דוח ביצועים מלא
report = classification_report(y_test_tensor.numpy(),
                               predictions.numpy(),
                               target_names=[f"Label_{i+1}" for i in range(4)])

# הצגת הדוח
print(report)


              precision    recall  f1-score   support

     Label_1       0.64      0.64      0.64        11
     Label_2       0.29      0.50      0.36         8
     Label_3       0.64      0.58      0.61        12
     Label_4       0.31      0.57      0.40         7

   micro avg       0.45      0.58      0.51        38
   macro avg       0.47      0.57      0.50        38
weighted avg       0.50      0.58      0.53        38
 samples avg       0.42      0.62      0.48        38



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
